In [215]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%pip install -q transformers
%pip install -q loguru
%pip install -q openai langchain langchain[docarray] tiktoken unstructured
%pip install -q llama-index==0.10.4 chromadb loguru pypdf  sentence_transformers llama-index-embeddings-huggingface
%pip install -q pdf2image  pdfminer.six
%pip install -q pillow-heif
%pip install -q unstructured-inference
%pip install -q pikepdf

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.2.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.2.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.2.0 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.2.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.1.1 requires torch<3.0.0,>=2.1.2, but you have torch 2.1.0 which is incompatible.


In [216]:
import openai

from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

In [217]:
!wget -O Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
FILE_NAME = "./Baidu-Inc-Code-of-Business-Conduct-and-Ethics.pdf"

--2024-02-14 16:21:27--  https://dl.dropbox.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
Resolving dl.dropbox.com (dl.dropbox.com)... 162.125.71.15, 2620:100:6021:15::a27d:410f
Connecting to dl.dropbox.com (dl.dropbox.com)|162.125.71.15|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876 [following]
--2024-02-14 16:21:28--  https://dl.dropboxusercontent.com/scl/fi/boxb3mvc0mv7d7tndmy0j/BAIDU-INC.-CODE-OF-BUSINESS-CONDUCT-AND-ETHICS.pdf?rlkey=gs378jtvi6gwzy0ncn0jx0876
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.70.15, 2620:100:6021:15::a27d:410f
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.70.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221928 (217K) [app

## Similarity K

In [218]:
K = 5

## LangChain

In [219]:
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader

from typing import Any, List
from rich.pretty import pprint

def pretty_print(title: str=None, content: Any=None):
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)

from IPython.display import display, HTML

In [184]:
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [185]:
#@title load data { vertical-output: true}
loader = UnstructuredPDFLoader(FILE_NAME)
docs = loader.load()

In [186]:
#@title splitting { vertical-output: true}
text_splitter = SentenceTransformersTokenTextSplitter()
texts = text_splitter.split_documents(docs)

In [187]:
#@title indexing { vertical-output: true}
emb = OpenAIEmbeddings(openai_api_key=openai.api_key)
vec_db = Chroma.from_documents(documents=texts, embedding=emb)

In [223]:
#@title LLM { vertical-output: true}
lc_model = ChatOpenAI(openai_api_key=openai.api_key, temperature=1.5)

In [224]:
#@title base-retriever { vertical-output: true}
base_retriever = vec_db.as_retriever(k=K)

In [225]:
#@title MultiQueryRetriever { vertical-output: true}
final_retriever = MultiQueryRetriever.from_llm(base_retriever, lc_model)

In [226]:
#@title QA chain { vertical-output: true}
tmpl = """
You are an assistant to answer a question from user with a context.

Context:
{context}

Question:
{question}

The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
"""
prompt = ChatPromptTemplate.from_template(tmpl)
chain = {"question": RunnablePassthrough(), "context": final_retriever} \
        | prompt \
        | lc_model \
        | StrOutputParser() \

result = chain.invoke("Waht is the doc talking about?")

display(HTML(result))

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the main topic discussed in the document?', '2. Could you provide a brief summary of the subject matter of the document?', '3. What does the document primarily cover and discuss?']


## Llama-Index

In [205]:
TEMPERATURE = 0.0
SIM_TOP_K = 3
RERANK_TOP_K = 3
WIN_SZ = 1024

In [220]:
import asyncio
from tqdm.asyncio import tqdm

import chromadb
from chromadb.api.models.Collection import Collection

from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.core import QueryBundle
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate
from llama_index.core.llms.utils import LLMType
from llama_index.core.schema import NodeWithScore
from llama_index.core.indices.base import BaseIndex
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.embeddings.utils import EmbedType
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.legacy.vector_stores import ChromaVectorStore
from llama_index.core.node_parser import SentenceWindowNodeParser

In [207]:
def create_service_context(llm: LLMType, embs: EmbedType) -> ServiceContext:
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=WIN_SZ,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )

    return ServiceContext.from_defaults(
        node_parser=node_parser,
        llm=llm,
        embed_model=embs,
    )

def create_vectors(path: str, collection_name="tmp_collection") -> Collection:
    chroma_client = chromadb.PersistentClient(path)
    # https://github.com/run-llama/llama_index/issues/6528
    return chroma_client.get_or_create_collection(collection_name)

def create_storage_context() -> StorageContext:
    path: str = "./db/LlamaIndexMultiVectorSummaryAgent"
    return StorageContext.from_defaults(
        vector_store=ChromaVectorStore(create_vectors(path=path))
    )


In [208]:
#@title load data { vertical-output: true}
input_files: List[str] = [FILE_NAME]
docs: SimpleDirectoryReader = SimpleDirectoryReader( input_files=input_files).load_data()

In [209]:
#@title LLM { vertical-output: true}
li_model = OpenAI(model="gpt-4-1106-preview", temperature=TEMPERATURE)

In [210]:
#@title splitting and indexing { vertical-output: true}
embs = "local:BAAI/bge-small-en-v1.5"

service_context: ServiceContext = (
    create_service_context(li_model, embs)
)

storage_context: StorageContext = (
    create_storage_context()
)

vector_index: BaseIndex = VectorStoreIndex.from_documents(
    docs,
    service_context=service_context,
    #storage_context=storage_context,
    show_progress=True,
)

<ipython-input-207-0e0cc94f5ebb>:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  return ServiceContext.from_defaults(


Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/160 [00:00<?, ?it/s]

In [211]:
#@title base-retriever { vertical-output: true}
base_retriever = vector_index.as_retriever(similarity_top_k=K)

In [221]:
#@title MultiQueriesRetriever { vertical-output: true}
class MultiQueriesRetriever(BaseRetriever):
    def __init__(self, base_retriever: BaseRetriever, model:OpenAI):
        self.template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines.
    Original question: {question}""")
        self._retrievers = [base_retriever]
        self.base_retriever = base_retriever
        self.model = model

    def gen_queries(self, query) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3-turbo", temperature=1.5)
        prompt = self.template.format(question=query)
        res = self.model.complete(prompt)
        return res.text.split("\n")

    async def run_gen_queries(self,generated_queries: List[str]) -> List[NodeWithScore]:
        tasks = list(map(lambda q: self.base_retriever.aretrieve(q), generated_queries))
        res = await tqdm.gather(*tasks)
        return res[0]

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        return list()

    async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query = query_bundle.query_str
        generated_queries = self.gen_queries(query)
        query_res = await self.run_gen_queries(generated_queries)
        return query_res



mr = MultiQueriesRetriever(base_retriever, li_model)


#ls = mr.gen_queries("Who is Bill Gates?")
#pretty_print("ls", ls)
#rls = await mr.run_gen_queries(ls)
#pretty_print("rls", rls)

In [222]:
#@title QA query { vertical-output: true}
query_text = """What is the topic of content?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in HTML with appropriate markup for clarity and organization.
    """
final_res = await RetrieverQueryEngine(mr).aquery(query_text)
display(HTML(final_res.response))


100%|██████████| 5/5 [00:00<00:00, 28.36it/s]
